In [1]:
"'IMPORTS TO BE USED IN THE LAB'"


# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn import metrics as mt
from skimage.io import imshow

from copy import deepcopy

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score , auc
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder


#preprocess
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import ShuffleSplit


#1D CNN Imports
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers.legacy import RMSprop
from keras.utils import FeatureSpace

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
import keras as keras
from keras.layers import Reshape, Input
from keras.layers import RandomFlip, RandomRotation
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import average
from keras.models import  Model
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.optimizers import Adam
from keras.layers import RandomBrightness, RandomContrast
from keras.layers import Dropout, Flatten,Activation, Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D


from tensorflow.keras.layers import Average
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Subtract
from tensorflow.keras.layers import Embedding, Input, Concatenate



import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2
import numpy as np
from tqdm import tqdm
import os
from random import shuffle
from zipfile import ZipFile
from PIL import Image

from scipy.stats import ttest_rel
from sklearn.model_selection import cross_val_score

# Package to Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# For Embeddings
from tensorflow.keras.layers import Embedding



print("TensorFlow version:", tf.__version__)
tf.config.list_physical_devices('GPU')

# from google.colab import drive
# drive.mount('/content/drive')

TensorFlow version: 2.15.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
df = pd.read_csv("./diabetes_risk_prediction_dataset.csv")

cat_labels = ['Gender','Polyuria','Polydipsia','sudden weight loss','weakness','Polyphagia','Genital thrush','visual blurring','Itching','Irritability','delayed healing','partial paresis','muscle stiffness','Alopecia','Obesity','class']
num_labels = ['Age']
NUM_CLASSES  = 2
my_target_list = ['Positive' , 'Negative']


print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   sudden weight loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital thrush      520 non-null    object
 8   visual blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed healing     520 non-null    object
 12  partial paresis     520 non-null    object
 13  muscle stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [3]:
"""HELPER FUNCTIONS"""
labels = [ ]
# Function to calculate fbeta_score
def fbeta_score(y_true, y_pred, beta=1): #OUR FBETA SCORE IS LESS THAN 1
    # Clip predictions
    y_pred = K.clip(y_pred, 0, 1)

    # Calculate precision and recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (actual_positives + K.epsilon())

    # Calculate F-beta score
    beta_squared = beta**2
    fbeta = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

    return K.mean(fbeta)

def summarize_net(net, X_test, y_test, title_text=''):
    plt.figure(figsize=(15, 5))
    yhat = np.argmax(net.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to 1d array
    acc = mt.accuracy_score(y_true, yhat)
    cm = mt.confusion_matrix(y_true, yhat)
    cm = cm / np.sum(cm, axis=1)[:, np.newaxis]
    sns.heatmap(cm, annot=True, fmt='.2f', xticklabels=labels, yticklabels=labels)
    plt.title(title_text + ' Accuracy: {:.4f}'.format(acc))


def setup_data(dataset , cat_cols):
    df_processed = dataset.copy()
    
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()
    
    for column in cat_cols:
        # Use label encoding for each categorical column
        df_processed[column] = label_encoder.fit_transform(dataset[column])

    return df_processed

batch_size = 64

def create_dataset_from_dataframe(df_input):

    df = df_input.copy()
    labels = df['class']

    df = {key: value.values[:,np.newaxis] for key, value in df_input[cat_labels+num_labels].items()}

    # create the Dataset here
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    
    # now enable batching and prefetching
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    
    return ds

    

Because we care creeating two different types of models, the data preprocessing will be different for each. To start out, we will just save the csv into a variable called 'df' using Pandas `read_csv()` function.

## First Model: Wide and Deep Network

One of the perks of using a Wide and Deep Network is that we can use cross-product features to improve our model, which implements "memorization" into our model by allowing it to identify relationships between some of the variables. To create the Wide and Deep Network, we will be using Keras FeatureSpaces to build out the Wide portion, and the we will be using Keras Model and Layers to create the model.

In [9]:
df_wd = deepcopy(df)
# Correcting Bool Variable Representation
mappings_for_target = {'Positive': True, 'Negative': False}
df_wd['class'] = df_wd['class'].map(mappings_for_target)


# Correcting Categorical Variable Representation
column_mappings = {}
for column in df_wd.drop(columns=['class']).columns:
    column_mapping = {category: index for index, category in enumerate(df_wd[column].unique())}
    column_mappings[column] = column_mapping
for column, column_mapping in column_mappings.items():
    df_wd[column] = df_wd[column].map(column_mapping)

shuffle_split = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in shuffle_split.split(df):
  df_train= df.iloc[train_index]
  df_test= df.iloc[test_index]

feature_space= FeatureSpace(
    features= {
        # "Age": FeatureSpace.float_discretized(num_bins=10),
        "Gender": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Polyuria": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Polydipsia": FeatureSpace.integer_categorical(num_oov_indices=0),
        "sudden weight loss": FeatureSpace.integer_categorical(num_oov_indices=0),
        "weakness": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Polyphagia": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Genital thrush": FeatureSpace.integer_categorical(num_oov_indices=0),
        "visual blurring": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Itching": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Irritability": FeatureSpace.integer_categorical(num_oov_indices=0),
        "delayed healing": FeatureSpace.integer_categorical(num_oov_indices=0),
        "partial paresis": FeatureSpace.integer_categorical(num_oov_indices=0),
        "muscle stiffness": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Alopecia": FeatureSpace.integer_categorical(num_oov_indices=0),
        "Obesity": FeatureSpace.integer_categorical(num_oov_indices=0),
    }, crosses=[
        FeatureSpace.cross(
            feature_names= ('Polyuria', 'Polydipsia'),
            crossing_dim= 2*2),
        FeatureSpace.cross(
            feature_names= ('Polyuria', 'sudden_weight_loss'),
            crossing_dim= 2*2),
        FeatureSpace.cross(
            feature_names= ('Polyuria', 'weakness'),
            crossing_dim= 2*2),
        FeatureSpace.cross(
            feature_names= ('Polyuria', 'Polyphagia'),
            crossing_dim= 2*2)

    ],
    output_mode="concat"
)

# Creating Tensors for train and test datasets
categorical_headers= df_wd.drop(columns=['class']).columns

ds_train= create_dataset_from_dataframe(df_train)
ds_test= create_dataset_from_dataframe(df_test)


print(ds_train)
# Performing One Hot Encoding
# ds_train_no_label = ds_train.map(lambda x, _: x)
# feature_space.adapt(ds_train_no_label)

train_ds_with_no_labels = ds_train_no_label.map(lambda x: x)
feature_space.adapt(train_ds_with_no_labels)

<_PrefetchDataset element_spec=({'Gender': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Polyuria': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Polydipsia': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'sudden weight loss': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'weakness': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Polyphagia': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Genital thrush': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'visual blurring': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Itching': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Irritability': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'delayed healing': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'partial paresis': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'muscle stiffness': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'Alopecia': TensorSp

UnimplementedError: Graph execution error:

Detected at node Cast defined at (most recent call last):
  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>

  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 619, in start

  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/marcoszertuche/anaconda3/envs/lab6env/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/Users/marcoszertuche/anacond

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.layers import Embedding, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model


def setup_embedding_from_crossing(feature_space, col_name):
    # what the maximum integer value for this variable?

    # get the size of the feature
    N = feature_space.crossers[col_name].num_bins
    x = feature_space.crossers[col_name].output


    # now use an embedding to deal with integers as if they were one hot encoded
    x = Embedding(input_dim=N,
                  output_dim=int(np.sqrt(N)),
                  input_length=1, name=col_name+'_embed')(x)

    x = Flatten()(x) # get rid of that pesky extra dimension (for time of embedding)

    return x


def setup_embedding_from_categorical(feature_space, col_name):
    # what the maximum integer value for this variable?
    # which is the same as the number of categories
    N = len(feature_space.preprocessors[col_name].get_vocabulary())

    # get the output from the feature space, which is input to embedding
    x = feature_space.preprocessors[col_name].output

    # now use an embedding to deal with integers from feature space
    x = Embedding(input_dim=N,
                  output_dim=int(np.sqrt(N)),
                  input_length=1, name=col_name+'_embed')(x)

    x = Flatten()(x) # get rid of that pesky extra dimension (for time of embedding)

    return x # return the tensor here


dict_inputs = feature_space.get_inputs() # need to use unprocessed features here, to gain access to each output

# we need to create separate lists for each branch
crossed_outputs = []

# for each crossed variable, make an embedding
for col in feature_space.crossers.keys():
    print()
    x = setup_embedding_from_crossing(feature_space, col)

    # save these outputs in list to concatenate later
    crossed_outputs.append(x)


# now concatenate the outputs and add a fully connected layer
wide_branch1 = Concatenate(name='wide_concat')(crossed_outputs)

# reset this input branch
all_deep_branch_outputs1 = []



# for each categorical variable
for col in categorical_headers:

    # get the output tensor from ebedding layer
    x = setup_embedding_from_categorical(feature_space, col)

    # save these outputs in list to concatenate later
    all_deep_branch_outputs1.append(x)


# merge the deep branches together
deep_branch1 = Concatenate(name='embed_concat1')(all_deep_branch_outputs1)
deep_branch1 = Dense(units=20,activation='relu', name='deep1_1')(deep_branch1)
deep_branch1 = Dense(units=5,activation='relu', name='deep1_2')(deep_branch1)

# merge the deep and wide branch
final_branch1 = Concatenate(name='concat_deep_wide1')([deep_branch1, wide_branch1])
final_branch1 = Dense(units=1,activation='sigmoid',
                     name='combined1')(final_branch1)

training_model1 = keras.Model(inputs=dict_inputs, outputs=final_branch1)
training_model1.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC(name='auc')]
)



# Now you can use it in model.fit
history1 = training_model1.fit(
    ds_train, epochs=5, validation_data=ds_test, verbose=2
)

#### Selection of FeatureSpace Crosses

After a bit of research into early indicators into diabetes, I chose my FeatureSpace crosses to match what was most commonly stated as an early symptom from the different resources. 

In [ ]:
df = setup_data(df , cat_labels)

df_data = df.drop(columns=['class'], axis = 1)

df_target = df['class']

# Split it into train / test subsets
X_train, X_test, y_train, y_test = train_test_split(df_data, df_target, test_size=0.2,
                                                            stratify=df_target, 
                                                            random_state=42)


# print some stats of the data
print("X_train Shape:",X_train.shape, "Label Shape:", y_train.shape)
uniq_classes = np.sum(y_train,axis=0)
plt.bar(list(range(NUM_CLASSES)),uniq_classes)
plt.xticks(list(range(NUM_CLASSES)), my_target_list, rotation='vertical')
plt.ylabel("Number of Instances in Each Class")
plt.show()

## Creating a Wide and Deep Network

## Creating a Convolutional Neural Network (CNN)